In [1]:
pwd

'/Users/abbystokes/Documents/GitHub/wildfire-risk-ml'

In [10]:
import os
import glob
import rasterio
import numpy as np

# Checking nans per band

merged_dir = 'data/merged'
merged_files = sorted(glob.glob(os.path.join(merged_dir, 'merged_ID*.tif')))

# Initialize accumulators
band_count = None
band_names = []
global_min = []
global_max = []
global_sum = []
global_sum_sq = []
global_nan_count = []
global_pixel_count = []

for merged_path in merged_files:
    with rasterio.open(merged_path) as src:
        if band_count is None:
            band_count = src.count
            band_names = list(src.descriptions) if src.descriptions else [f'Band {i+1}' for i in range(band_count)]
            global_min = [np.inf] * band_count
            global_max = [-np.inf] * band_count
            global_sum = [0.0] * band_count
            global_sum_sq = [0.0] * band_count
            global_nan_count = [0] * band_count
            global_pixel_count = [0] * band_count

        for i in range(1, src.count + 1):
            band = src.read(i).astype(np.float32)
            nodata = src.nodata
            if nodata is not None:
                band[band == nodata] = np.nan

            valid_pixels = ~np.isnan(band)
            band_valid = band[valid_pixels]

            if band_valid.size > 0:
                idx = i - 1
                global_min[idx] = min(global_min[idx], np.nanmin(band_valid))
                global_max[idx] = max(global_max[idx], np.nanmax(band_valid))
                global_sum[idx] += np.nansum(band_valid)
                global_sum_sq[idx] += np.nansum(band_valid**2)
                global_nan_count[idx] += np.isnan(band).sum()
                global_pixel_count[idx] += band_valid.size

# Final stats
global_mean = [s / c if c > 0 else np.nan for s, c in zip(global_sum, global_pixel_count)]
global_std = [
    np.sqrt((sq / c) - (m ** 2)) if c > 0 else np.nan
    for sq, c, m in zip(global_sum_sq, global_pixel_count, global_mean)
]

# Print summary
print("\n📊 Overall Summary Statistics Across All Merged Bands:")
for i in range(band_count):
    name = band_names[i] if band_names[i] else f"Band {i+1}"
    print(f"{name}:")
    print(f"  Min: {global_min[i]:.4f}")
    print(f"  Max: {global_max[i]:.4f}")
    print(f"  Mean: {global_mean[i]:.4f}")
    print(f"  Std: {global_std[i]:.4f}")
    print(f"  NaN count: {global_nan_count[i]}")
    print(f"  Total valid pixels: {global_pixel_count[i]}")
    print("Percent nan:", global_nan_count[i]/global_pixel_count[i])



📊 Overall Summary Statistics Across All Merged Bands:
Ignition Point:
  Min: 0.0000
  Max: 0.0087
  Mean: 0.0000
  Std: 0.0001
  NaN count: 0
  Total valid pixels: 250871808
Percent nan: 0.0
Distance to Water:
  Min: 0.0000
  Max: 32871.3164
  Mean: 5813.8085
  Std: 3881.6862
  NaN count: 8292923
  Total valid pixels: 242578885
Percent nan: 0.03418649978542032
Heat Load:
  Min: 0.0000
  Max: 1.0000
  Mean: 0.5343
  Std: 0.3278
  NaN count: 8493155
  Total valid pixels: 242378653
Percent nan: 0.035040854030985974
Topographic Complexity:
  Min: 0.0000
  Max: 23.2847
  Mean: 2.2723
  Std: 2.1707
  NaN count: 8645889
  Total valid pixels: 242225919
Percent nan: 0.03569349240450193
Slope:
  Min: 0.0000
  Max: 64.3769
  Mean: 7.2479
  Std: 8.4056
  NaN count: 8493155
  Total valid pixels: 242378653
Percent nan: 0.035040854030985974
Elevation:
  Min: 0.0000
  Max: 3182.5840
  Mean: 358.5952
  Std: 395.6938
  NaN count: 8429162
  Total valid pixels: 242442646
Percent nan: 0.0347676538722482
A

In [6]:
import numpy as np
import rasterio
import os
import glob

merged_dir = 'data/merged'
merged_files = sorted(glob.glob(os.path.join(merged_dir, 'merged_ID*.tif')))

# Band names to clean and threshold for invalid values
target_band_names = ['wind speed', 'vapor pressure']
negative_threshold = -1000
nodata_value = -9999  # Replace NaNs with this when saving

for merged_path in merged_files:
    with rasterio.open(merged_path) as src:
        profile = src.profile.copy()
        descriptions = src.descriptions
        cleaned_bands = []

        for i in range(1, src.count + 1):
            band = src.read(i).astype(np.float32)
            desc = descriptions[i - 1].lower() if descriptions[i - 1] else f"Band {i}"

            if any(keyword in desc for keyword in target_band_names):
                invalid_mask = band < negative_threshold
                band[invalid_mask] = np.nan
                print(f"{os.path.basename(merged_path)} - '{desc}': set {invalid_mask.sum()} values to NaN")

            # Replace NaNs with nodata value for saving
            band[np.isnan(band)] = nodata_value
            cleaned_bands.append(band)

    # Update profile
    profile.update(dtype=rasterio.float32, nodata=nodata_value)

    # Overwrite original file
    with rasterio.open(merged_path, 'w', **profile) as dst:
        for i, band in enumerate(cleaned_bands, start=1):
            dst.write(band, i)
            if descriptions[i - 1]:
                dst.set_band_description(i, descriptions[i - 1])



merged_ID1.tif - 'water vapor pressure': set 0 values to NaN
merged_ID1.tif - 'wind speed': set 0 values to NaN
merged_ID100000.tif - 'water vapor pressure': set 0 values to NaN
merged_ID100000.tif - 'wind speed': set 0 values to NaN
merged_ID100027.tif - 'water vapor pressure': set 0 values to NaN
merged_ID100027.tif - 'wind speed': set 0 values to NaN
merged_ID100032.tif - 'water vapor pressure': set 0 values to NaN
merged_ID100032.tif - 'wind speed': set 0 values to NaN
merged_ID100041.tif - 'water vapor pressure': set 0 values to NaN
merged_ID100041.tif - 'wind speed': set 0 values to NaN
merged_ID100057.tif - 'water vapor pressure': set 99771 values to NaN
merged_ID100057.tif - 'wind speed': set 99771 values to NaN
merged_ID100058.tif - 'water vapor pressure': set 0 values to NaN
merged_ID100058.tif - 'wind speed': set 0 values to NaN
merged_ID100081.tif - 'water vapor pressure': set 0 values to NaN
merged_ID100081.tif - 'wind speed': set 0 values to NaN
merged_ID100090.tif - 'wat

In [11]:
### Code to imput NAN values

import os
import glob
import rasterio
import numpy as np

merged_dir = 'data/merged'
merged_files = sorted(glob.glob(os.path.join(merged_dir, 'merged_ID*.tif')))

# ---------- First Pass: Calculate Global Stats ----------
band_count = None
band_names = []
global_min = []
global_max = []
global_sum = []
global_sum_sq = []
global_nan_count = []
global_pixel_count = []

for merged_path in merged_files:
    with rasterio.open(merged_path) as src:
        if band_count is None:
            band_count = src.count
            band_names = list(src.descriptions) if src.descriptions else [f'Band {i+1}' for i in range(band_count)]
            global_min = [np.inf] * band_count
            global_max = [-np.inf] * band_count
            global_sum = [0.0] * band_count
            global_sum_sq = [0.0] * band_count
            global_nan_count = [0] * band_count
            global_pixel_count = [0] * band_count

        for i in range(1, src.count + 1):
            band = src.read(i).astype(np.float32)
            nodata = src.nodata
            if nodata is not None:
                band[band == nodata] = np.nan

            valid_pixels = ~np.isnan(band)
            band_valid = band[valid_pixels]

            if band_valid.size > 0:
                idx = i - 1
                global_min[idx] = min(global_min[idx], np.nanmin(band_valid))
                global_max[idx] = max(global_max[idx], np.nanmax(band_valid))
                global_sum[idx] += np.nansum(band_valid)
                global_sum_sq[idx] += np.nansum(band_valid**2)
                global_nan_count[idx] += np.isnan(band).sum()
                global_pixel_count[idx] += band_valid.size

global_mean = [s / c if c > 0 else np.nan for s, c in zip(global_sum, global_pixel_count)]

# ---------- Second Pass: Replace NaNs and Overwrite Files ----------
nodata_value = -9999

for merged_path in merged_files:
    with rasterio.open(merged_path) as src:
        profile = src.profile.copy()
        descriptions = src.descriptions
        cleaned_bands = []

        for i in range(1, src.count + 1):
            band = src.read(i).astype(np.float32)
            desc = descriptions[i - 1].lower() if descriptions[i - 1] else f"band {i}"
            idx = i - 1

            # Convert nodata to NaN
            if src.nodata is not None:
                band[band == src.nodata] = np.nan

            nan_mask = np.isnan(band)

            if "time since last fire" in desc:
                band[nan_mask] = 33
                print(f"{os.path.basename(merged_path)} - '{desc}': Replaced {nan_mask.sum()} NaNs with 33")
            else:
                band[nan_mask] = global_mean[idx]
                print(f"{os.path.basename(merged_path)} - '{desc}': Replaced {nan_mask.sum()} NaNs with global mean {global_mean[idx]:.2f}")

            cleaned_bands.append(band)

    # Update profile
    profile.update(dtype=rasterio.float32, nodata=nodata_value)

    with rasterio.open(merged_path, 'w', **profile) as dst:
        for i, band in enumerate(cleaned_bands, start=1):
            dst.write(band, i)
            if descriptions[i - 1]:
                dst.set_band_description(i, descriptions[i - 1])


merged_ID1.tif - 'ignition point': Replaced 0 NaNs with global mean 0.00
merged_ID1.tif - 'distance to water': Replaced 0 NaNs with global mean 5813.81
merged_ID1.tif - 'heat load': Replaced 0 NaNs with global mean 0.53
merged_ID1.tif - 'topographic complexity': Replaced 0 NaNs with global mean 2.27
merged_ID1.tif - 'slope': Replaced 0 NaNs with global mean 7.25
merged_ID1.tif - 'elevation': Replaced 0 NaNs with global mean 358.60
merged_ID1.tif - 'aspect': Replaced 0 NaNs with global mean 199.70
merged_ID1.tif - 'ndvi': Replaced 0 NaNs with global mean 0.45
merged_ID1.tif - 'fire history': Replaced 0 NaNs with global mean 0.00
merged_ID1.tif - 'time since last fire': Replaced 261789 NaNs with 33
merged_ID1.tif - 'max temperature anomaly': Replaced 0 NaNs with global mean 0.18
merged_ID1.tif - 'minimum temperature anomaly': Replaced 0 NaNs with global mean 0.18
merged_ID1.tif - 'precipitation anomaly': Replaced 0 NaNs with global mean -0.81
merged_ID1.tif - 'accumulated annual precipit

In [12]:
# Now lets check nans again
merged_dir = 'data/merged'
merged_files = sorted(glob.glob(os.path.join(merged_dir, 'merged_ID*.tif')))

# Initialize accumulators
band_count = None
band_names = []
global_min = []
global_max = []
global_sum = []
global_sum_sq = []
global_nan_count = []
global_pixel_count = []

for merged_path in merged_files:
    with rasterio.open(merged_path) as src:
        if band_count is None:
            band_count = src.count
            band_names = list(src.descriptions) if src.descriptions else [f'Band {i+1}' for i in range(band_count)]
            global_min = [np.inf] * band_count
            global_max = [-np.inf] * band_count
            global_sum = [0.0] * band_count
            global_sum_sq = [0.0] * band_count
            global_nan_count = [0] * band_count
            global_pixel_count = [0] * band_count

        for i in range(1, src.count + 1):
            band = src.read(i).astype(np.float32)
            nodata = src.nodata
            if nodata is not None:
                band[band == nodata] = np.nan

            valid_pixels = ~np.isnan(band)
            band_valid = band[valid_pixels]

            if band_valid.size > 0:
                idx = i - 1
                global_min[idx] = min(global_min[idx], np.nanmin(band_valid))
                global_max[idx] = max(global_max[idx], np.nanmax(band_valid))
                global_sum[idx] += np.nansum(band_valid)
                global_sum_sq[idx] += np.nansum(band_valid**2)
                global_nan_count[idx] += np.isnan(band).sum()
                global_pixel_count[idx] += band_valid.size

# Final stats
global_mean = [s / c if c > 0 else np.nan for s, c in zip(global_sum, global_pixel_count)]
global_std = [
    np.sqrt((sq / c) - (m ** 2)) if c > 0 else np.nan
    for sq, c, m in zip(global_sum_sq, global_pixel_count, global_mean)
]

# Print summary
print("\n📊 Overall Summary Statistics Across All Merged Bands:")
for i in range(band_count):
    name = band_names[i] if band_names[i] else f"Band {i+1}"
    print(f"{name}:")
    print(f"  Min: {global_min[i]:.4f}")
    print(f"  Max: {global_max[i]:.4f}")
    print(f"  Mean: {global_mean[i]:.4f}")
    print(f"  Std: {global_std[i]:.4f}")
    print(f"  NaN count: {global_nan_count[i]}")
    print(f"  Total valid pixels: {global_pixel_count[i]}")
    print("Percent nan:", global_nan_count[i]/global_pixel_count[i])



📊 Overall Summary Statistics Across All Merged Bands:
Ignition Point:
  Min: 0.0000
  Max: 0.0087
  Mean: 0.0000
  Std: 0.0001
  NaN count: 0
  Total valid pixels: 250871808
Percent nan: 0.0
Distance to Water:
  Min: 0.0000
  Max: 32871.3164
  Mean: 5813.8086
  Std: 3816.9896
  NaN count: 0
  Total valid pixels: 250871808
Percent nan: 0.0
Heat Load:
  Min: 0.0000
  Max: 1.0000
  Mean: 0.5343
  Std: 0.3222
  NaN count: 0
  Total valid pixels: 250871808
Percent nan: 0.0
Topographic Complexity:
  Min: 0.0000
  Max: 23.2847
  Mean: 2.2723
  Std: 2.1330
  NaN count: 0
  Total valid pixels: 250871808
Percent nan: 0.0
Slope:
  Min: 0.0000
  Max: 64.3769
  Mean: 7.2479
  Std: 8.2621
  NaN count: 0
  Total valid pixels: 250871808
Percent nan: 0.0
Elevation:
  Min: 0.0000
  Max: 3182.5840
  Mean: 358.5952
  Std: 388.9895
  NaN count: 0
  Total valid pixels: 250871808
Percent nan: 0.0
Aspect:
  Min: 0.0000
  Max: 359.9471
  Mean: 199.6984
  Std: 92.1633
  NaN count: 0
  Total valid pixels: 25087

In [ ]:
#Nice! No more nans. :)